In [1]:
from tool import *
import plotly.graph_objects as go

In [2]:
with open('D:/Data/Shipment Plan_Adjusted_new.DB', 'rb') as f:
    sp = pickle.load(f)

# 1. 신규 생산계획 불러와 데이터 가공

In [4]:
pp = pd.read_excel(get_filename(), sheet_name=0, skiprows=1)

C:\Users\paul76.lee\AppData\Local\Temp\ipykernel_20332\3045562759.py:1: FutureWarning: Inferring datetime64[ns] from data containing strings is deprecated and will be removed in a future version. To retain the old behavior explicitly pass Series(data, dtype=datetime64[ns])
  pp = pd.read_excel(filepath, sheet_name=0, skiprows=1)


In [5]:
pp_date = []
for item in pp.columns:
    if type(item) == datetime.datetime:
        pp_date.append(item)

pp = pd.concat([pp.iloc[:, :7], pp[pp_date]], axis=1)
pp.columns = ['Target Week', 'ODF', 'TCL BOM', 'drop3', 'drop4', 'Production Date', 'WO'] + pp_date
pp = pp[['Target Week', 'ODF', 'TCL BOM', 'Production Date', 'WO'] + pp_date]
pp = pp.set_index(['Target Week', 'ODF', 'TCL BOM'])[pp_date].stack().reset_index()
pp.columns = ['Target Week', 'ODF', 'TCL BOM', 'PP Date', 'QTY']
pp

In [12]:
pp = pp[pp['QTY'].apply(lambda x:type(x) in [int, float])]
pp['QTY'] = pp['QTY'].astype(int)
pp['Rep PMS'] = pp['TCL BOM'].map(tcl_bom_map)
pp['Mapping Model.Suffix'] = pp['Rep PMS'].map(make_dict_from_df(sp[['Rep PMS', 'Mapping Model.Suffix']].drop_duplicates())).fillna('TBD')
pp['Updated_at'] = datetime.datetime(2023,12,25,22,39) # 업데이트 일자

C:\Users\paul76.lee\AppData\Local\Temp\ipykernel_20332\2498862157.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pp['QTY'] = pp['QTY'].astype(int)
C:\Users\paul76.lee\AppData\Local\Temp\ipykernel_20332\2498862157.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pp['Rep PMS'] = pp['TCL BOM'].map(tcl_bom_map)
C:\Users\paul76.lee\AppData\Local\Temp\ipykernel_20332\2498862157.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_

In [13]:
pp[pp['Rep PMS'].isnull()] # Rep PMS 명이 null인것 있는지 확인

,Target Week,ODF,TCL BOM,PP Date,QTY,Rep PMS,Mapping Model.Suffix,Updated_at


In [14]:
pp['Mapping Model.Suffix'].unique() # Mapping Model Suffix에 이상값 있는지 확인

array(['22MR410-B.ATIQ', '24MR400-B.AMAQ', '24MR400-B.AWPQ',
       '22MR410-B.ATVQ', '27MR400-B.ATVQ', '27MR400-B.ATMQ',
       '27MR400-B.AWPQ', '22MR410-B.AUSQ', '22MR410-B.APDQ',
       '22MR410-B.AWPQ', '22MR410-B.AHKQ', '22MR410.AKRQ',
       '24MR400-B.ATRQ', '27MR400-B.ATSQ', '27MR400-B.ATRQ',
       '27MR400-B.APHQ', '27MR400-B.APDQ', '24MR400-B.ATSQ',
       '24MR400-B.AMIQ', '22MR410-B.APHQ', '22MR410-B.ATRQ',
       '27MR400-B.AMIQ', '22MR410-B.ATMQ', '27MR400-B.ATIQ',
       '22MR410-B.AWHQ', '27MR400-B.AWHQ', '24MR400-B.AAUQ',
       '27MR400-B.AAUQ', '24MR400-B.APHQ', '22MR410-B.AEKQ',
       '22MR410-B.AMAQ', '27MR400-B.AEUQ', '27MR400-B.AEKQ',
       '24MR400-B.APDQ', '24MR400-B.AHKQ', '22MR41A-B.ACCQ',
       '22MR41S-B.ACCQ', '27MR400.AKRQ', '27MR41A-B.ACCQ',
       '27MR41S-B.ACCQ', '22MR410-B.AEUQ', '27MR400-B.AHKQ',
       '27MR400-B.AUSQ', '27MR400-B.AJLQ', '27MR400-B.AMAQ',
       '22MR410-B.AAUQ', '22MR410-B.AWMQ', '24MR400-B.AMFQ',
       '27MC41D-BS.ACCQ', '2

In [27]:
c1 = pp['TCL BOM'] == 'X3-27M205M3-USS0EU' # Costco 2 pack 수량을 outer box 기준으로 재조정

In [34]:
pp[c1] 

,Target Week,ODF,TCL BOM,PP Date,QTY,Rep PMS,Mapping Model.Suffix,Updated_at
1345,24/01/01W,DOK7517275,X3-27M205M3-USS0EU,2023-12-25,80,27MC41D-BS.CCCRMV,27MC41D-BS.ACCQ,2023-12-25 22:39:00
1376,24/01/15W,DOK7517801,X3-27M205M3-USS0EU,2024-01-10,1200,27MC41D-BS.CCCRMV,27MC41D-BS.ACCQ,2023-12-25 22:39:00
1400,24/01/22W,NaN,X3-27M205M3-USS0EU,2024-01-19,512,27MC41D-BS.CCCRMV,27MC41D-BS.ACCQ,2023-12-25 22:39:00
1458,24/02/18W,NaN,X3-27M205M3-USS0EU,2024-02-08,1280,27MC41D-BS.CCCRMV,27MC41D-BS.ACCQ,2023-12-25 22:39:00


In [33]:
pp.loc[c1, 'QTY'] = pp.loc[c1, 'QTY'] / 2

In [35]:
pp.shape

(294, 8)

In [36]:
pp = pp[pp['Rep PMS'].notnull()]

In [41]:
pp.shape

(294, 8)

In [45]:
c1 = pp['PP Date'] >= '2023-12-18'
c2 = pp['PP Date'] <= '2023-12-24'

In [46]:
pp[c1 & c2]

,Target Week,ODF,TCL BOM,PP Date,QTY,Rep PMS,Mapping Model.Suffix,Updated_at
1310,12/25W,DOK7517222,X3-24M205M3-INS0AU,2023-12-18,720,24MR400-BA.CTRRMV,24MR400-B.ATRQ,2023-12-25 22:39:00
1311,12/25W,DOK7517227,X3-22M205M3-AAS0AU,2023-12-18,252,22MR410-BB.CAURMV,22MR410-B.AAUQ,2023-12-25 22:39:00
1312,12/25W,DOK7517253,X3-22M205M3-MXS0AU,2023-12-19,840,22MR410-BB.CWMRMV,22MR410-B.AWMQ,2023-12-25 22:39:00
1314,12/25W,DOK7517225,X3-27M205M3-TKS0AU,2023-12-18,255,27MR400-BB.CPDRMV,27MR400-B.APDQ,2023-12-25 22:39:00
1315,12/25W,DOK7517220,X3-27M205M3-EUS0AU,2023-12-18,500,27MR400-BB.CEURMV,27MR400-B.AEUQ,2023-12-25 22:39:00
1316,12/25W,DOK7517254,X3-27M205M3-VMS0AU,2023-12-18,512,27MR400-BB.CTVRMV,27MR400-B.ATVQ,2023-12-25 22:39:00
1317,12/25W,DOK7517229,X3-27M205M3-AAS0AU,2023-12-18,560,27MR400-BB.CAURMV,27MR400-B.AAUQ,2023-12-25 22:39:00
1318,24/01/01W,DOK7517232,X3-22M205M3-AES0AU,2023-12-18,1680,22MR410-BB.CMARMV,22MR410-B.AMAQ,2023-12-25 22:39:00
1319,24/01/01W,DOK7517375,X3-22M205M3-UKS0AU,2023-12-23,1680,22MR410-BB.CEKRMV,22MR410-B.AEKQ,2023-12-25 22:39:00
1320,24/01/01W,DOK7517384,X3-22M205M3-CLS0AU,2023-12-23,741,22MR410-BB.CWHRMV,22MR410-B.AWHQ,2023-12-25 22:39:00


In [47]:
with open('D:/Data/TCL PP_result.bin', 'rb') as f:
    ppr_db = pickle.load(f)

In [52]:
ppr_db = pd.concat([ppr_db, pp[c1 & c2]]).reset_index(drop=True)

In [53]:
with open('D:Data/TCL PP_result.bin', 'wb') as f: # PP reulst에 저장
    pickle.dump(ppr_db, f)

In [54]:
pp.shape

(294, 8)

In [55]:
pp = pp[pp['PP Date'] >= '2023-12-25'].reset_index(drop=True)

In [56]:
pp.shape

(128, 8)

In [57]:
with open('D:/Data/TCL PP_new.bin', 'rb') as f:
    pp_db = pickle.load(f)

In [58]:
pp_db

,Target Week,Country,ODF,TCL BOM,Mapping Model.Suffix,Rep PMS,Production Date,WO,PP Date,QTY,Updated_at
0,NaN,NaN,DOK7516345,X3-24M205M3-AES0AU,24MR400-B.AMAQ,24MR400-BA.CMARMV,NaT,2400.0,2023-10-19,2200,2023-10-17 00:36:00
1,NaN,NaN,DOK7516345,X3-24M205M3-AES0AU,24MR400-B.AMAQ,24MR400-BA.CMARMV,NaT,2400.0,2023-10-20,200,2023-10-17 00:36:00
2,NaN,NaN,DOK7516364,X3-24M205M3-ARS0AU,24MR400-B.AMIQ,24MR400-BA.CMIRMV,NaT,1440.0,2023-10-20,1440,2023-10-17 00:36:00
3,NaN,NaN,DOK7516348,X3-24M205M3-AES0AU,24MR400-B.AMAQ,24MR400-BA.CMARMV,NaT,2400.0,2023-10-20,400,2023-10-17 00:36:00
4,NaN,NaN,DOK7516348,X3-24M205M3-AES0AU,24MR400-B.AMAQ,24MR400-BA.CMARMV,NaT,2400.0,2023-10-21,2000,2023-10-17 00:36:00
...,...,...,...,...,...,...,...,...,...,...,...
1324,24/02/05W,NaN,NaN,X3-27M205M3-ISS0AU,27MR400-B.AMFQ,27MR400-BB.CMFRMV,NaT,NaN,2024-02-02,512,2023-12-18 17:19:00
1325,24/02/05W,NaN,NaN,X3-27M205M3-KRS0AU,27MR400.AKRQ,27MR400-BB.CKRRMV,NaT,NaN,2024-02-03,1280,2023-12-18 17:19:00
1326,24/02/05W,NaN,NaN,X3-27M205M3-THS0AU,27MR400-B.ATMQ,27MR400-BB.CTMRMV,NaT,NaN,2024-02-03,256,2023-12-18 17:19:00
1327,24/02/05W,NaN,NaN,X3-27M205M3-UKS0AU,27MR400-B.AEKQ,27MR400-BB.CEKRMV,NaT,NaN,2024-02-03,1024,2023-12-18 17:19:00


In [63]:
pp_db = pd.concat([pp_db, pp]).reset_index(drop=True)

In [68]:
pp_db.shape

(1457, 11)

In [69]:
sorted(pp_db[pp_db['Updated_at'] == pp_db['Updated_at'].max()]['PP Date'].unique()) # 신규 PP의 생산일자가 맞게 되었는 지 확인

[numpy.datetime64('2023-12-25T00:00:00.000000000'),
 numpy.datetime64('2023-12-26T00:00:00.000000000'),
 numpy.datetime64('2023-12-27T00:00:00.000000000'),
 numpy.datetime64('2023-12-28T00:00:00.000000000'),
 numpy.datetime64('2023-12-29T00:00:00.000000000'),
 numpy.datetime64('2023-12-30T00:00:00.000000000'),
 numpy.datetime64('2024-01-02T00:00:00.000000000'),
 numpy.datetime64('2024-01-03T00:00:00.000000000'),
 numpy.datetime64('2024-01-04T00:00:00.000000000'),
 numpy.datetime64('2024-01-06T00:00:00.000000000'),
 numpy.datetime64('2024-01-08T00:00:00.000000000'),
 numpy.datetime64('2024-01-09T00:00:00.000000000'),
 numpy.datetime64('2024-01-10T00:00:00.000000000'),
 numpy.datetime64('2024-01-11T00:00:00.000000000'),
 numpy.datetime64('2024-01-12T00:00:00.000000000'),
 numpy.datetime64('2024-01-13T00:00:00.000000000'),
 numpy.datetime64('2024-01-15T00:00:00.000000000'),
 numpy.datetime64('2024-01-16T00:00:00.000000000'),
 numpy.datetime64('2024-01-17T00:00:00.000000000'),
 numpy.datet

In [70]:
# 업데이트된 pp_db를 저장
with open('D:/Data/TCL PP_new.bin', 'wb') as f:
    pickle.dump(pp_db, f)